In [1]:
import numpy as np
import pandas as pd
import time
import gezi
from gezi import tqdm
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from src import util

In [2]:
np.random.seed(2022)

In [3]:
df = pd.read_feather('../input/feedback-prize-2021/train.fea')

In [4]:
MAX_LEN = 2000
id = None
fes = []
fe = {}
pre_idx = 0
for i, row in tqdm(enumerate(df.itertuples()), total=len(df)):
  row = row._asdict()
  if row['id'] != id:
    if fe:
      for j in range(pre_idx, len(fe['text'].split())):
        fe['para_types'][j] = 0
      fe['para_types'] = fe['para_types'][:j + 1]
      fe['start'] = fe['start'][:j + 1]
      assert len(fe['para_types']) == len(fe['text'].split())
      assert min(fe['para_types']) >= 0
      fes.append(fe)
    fe = {}
    fe['id'] = row['id']
    fe['para_types'] = [-1] * MAX_LEN
    fe['start'] = [0] * MAX_LEN
    fe['text'] = row['text_']
    pre_idx = 0
    id = row['id']
  predictionstring = row['predictionstring']
  positions = predictionstring.split()
  start, end = int(positions[0]), int(positions[-1])
  fe['start'][start] = 1
  if start > pre_idx:
    for j in range(pre_idx, start):
      fe['para_types'][j] = 0
  for j in range(start, end + 1):
    fe['para_types'][j] =  row['discourse_type_id']
  pre_idx = end + 1
if fe:
  for j in range(pre_idx, len(fe['text'].split())):
    fe['para_types'][j] = 0
  fe['para_types'] = fe['para_types'][:j + 1]
  fe['start'] = fe['start'][:j + 1]
  assert len(fe['para_types']) == len(fe['text'].split())
  assert min(fe['para_types']) >= 0, fe['id']
  fes.append(fe)

  0%|          | 0/144293 [00:00<?, ?it/s]

In [5]:
d = pd.DataFrame(fes)

In [6]:
d

,id,para_types,start,text
0,423A1CA112E2,"[0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Phones\n\nModern humans today are always on th...
1,A8445CABFECE,"[0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Phones & Driving\n\nDrivers should not be able...
2,6B4F7A0165B9,"[0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Cell Phone Operation While Driving\n\nThe abil...
3,E05C7F5C1156,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",People are debating whether if drivers should ...
4,50B3435E475B,"[0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Texting and driving\n\nOver half of drivers in...
...,...,...,...,...
15589,8F4B595CF9E7,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Do you ever want more opinions and options whe...
15590,6B5809C83978,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Has anyone ever gave you advice? Was the advic...
15591,408A7D3D2EEC,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Imagine seeking advice from multiple people an...
15592,AFEC37C2D43F,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",There has been at least one point in everyone'...


In [7]:
d['part'] = [np.random.randint(19) for _ in range(len(d))]

In [8]:
gezi.init_flags()

In [9]:
def find_span(words, start, end):
  start2, end2 = None, None
  idx = 0
  for i, word in enumerate(words):
    if word == '\n':
      continue
    else:
      if idx == start:
        start2 = i
      if idx == end:
        end2 = i
      idx += 1
  return start2, end2

fes = []
for row in tqdm(d.itertuples(), total=len(d)):
  row = row._asdict()
  para_id = 0
  pre_type = None
  words = util.get_words(row['text'])
  pre_idx =0
  for i, para_type in enumerate(row['para_types']):
    if (para_type != pre_type) or (row['start'][i] == 1):
      if pre_type != None:
        fe = {}
        fe['id'] = row['id']
        fe['part'] = row['part']
        fe['para_id'] = para_id
        fe['para_type'] = pre_type
        fe['start'] = pre_idx
        fe['end'] = i
        start2, end2 = find_span(words, pre_idx, i)
        fe['start2'] = start2
        fe['end2'] = end2
        fe['para'] = ' '.join(words[start2: end2])
        fes.append(fe)
        para_id += 1
        pre_idx = i
      pre_type = para_type
  i += 1
  fe = {}
  fe['id'] = row['id']
  fe['part'] = row['part']
  fe['para_id'] = para_id
  fe['para_type'] = pre_type
  fe['start'] = pre_idx
  fe['end'] = i
  start2, end2 = find_span(words, pre_idx, i)
  fe['start2'] = start2
  fe['end2'] = end2
  fe['para'] = ' '.join(words[start2: end2])
  fes.append(fe)
      

  0%|          | 0/15594 [00:00<?, ?it/s]

In [10]:
d = pd.DataFrame(fes)

In [11]:
d

,id,part,para_id,para_type,start,end,start2,end2,para
0,423A1CA112E2,13,0,0,0,1,0,1.0,Phones
1,423A1CA112E2,13,1,5,1,45,1,45.0,[BR] [BR] Modern humans today are always on th...
2,423A1CA112E2,13,2,3,45,60,45,60.0,while driving. They are some really bad conseq...
3,423A1CA112E2,13,3,2,60,76,60,76.0,a phone. Some certain areas in the United Stat...
4,423A1CA112E2,13,4,2,76,139,76,139.0,"of it. [BR] [BR] When people have phones, they..."
...,...,...,...,...,...,...,...,...,...
175155,4C471936CD75,7,10,2,576,828,576,828.0,"decision. [BR] [BR] In my experience, seeking ..."
175156,4C471936CD75,7,11,3,828,839,828,839.0,regular class. This shows clearly that it is b...
175157,4C471936CD75,7,12,2,839,903,839,903.0,multiple opinions instead of just one. The imp...
175158,4C471936CD75,7,13,0,903,905,903,905.0,"someone's life,"


In [12]:
ids = []
texts = []
id = None
words = []
for row in tqdm(d.itertuples(), total=len(d)):
  row = row._asdict()
  if row['id'] != id:
    if words:
      ids.append(id)
      texts.append(' '.join(words))
    words = []
    id = row['id']
  words.extend(row['para'].split(' '))
if words:
  ids.append(id)
  texts.append(' '.join(words))
d_text = pd.DataFrame({
  'id': ids,
  'text': texts
})
d_text['text'] = d_text.text.apply(lambda x: x.replace('\n', '[SEP]'))

  0%|          | 0/175160 [00:00<?, ?it/s]

In [13]:
d = pd.merge(d, d_text, on=['id'], how='left')
d['pid'] = range(len(d))
d['para'] = d.para.apply(lambda x: x.replace('\n', '[SEP]'))

In [14]:
d = pd.merge(d, df[['id', 'kfold']].drop_duplicates(), on='id', how='inner')

In [15]:
d['worker'] = d['part'] * 5 + d['kfold']

In [16]:
d

,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,worker
0,423A1CA112E2,13,0,0,0,1,0,1.0,Phones,Phones [BR] [BR] Modern humans today are alway...,0,0,65
1,423A1CA112E2,13,1,5,1,45,1,45.0,[BR] [BR] Modern humans today are always on th...,Phones [BR] [BR] Modern humans today are alway...,1,0,65
2,423A1CA112E2,13,2,3,45,60,45,60.0,while driving. They are some really bad conseq...,Phones [BR] [BR] Modern humans today are alway...,2,0,65
3,423A1CA112E2,13,3,2,60,76,60,76.0,a phone. Some certain areas in the United Stat...,Phones [BR] [BR] Modern humans today are alway...,3,0,65
4,423A1CA112E2,13,4,2,76,139,76,139.0,"of it. [BR] [BR] When people have phones, they...",Phones [BR] [BR] Modern humans today are alway...,4,0,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175155,4C471936CD75,7,10,2,576,828,576,828.0,"decision. [BR] [BR] In my experience, seeking ...","In ancient times, and also still today in some...",175155,2,37
175156,4C471936CD75,7,11,3,828,839,828,839.0,regular class. This shows clearly that it is b...,"In ancient times, and also still today in some...",175156,2,37
175157,4C471936CD75,7,12,2,839,903,839,903.0,multiple opinions instead of just one. The imp...,"In ancient times, and also still today in some...",175157,2,37
175158,4C471936CD75,7,13,0,903,905,903,905.0,"someone's life,","In ancient times, and also still today in some...",175158,2,37


In [17]:
# d.to_csv('../input/feedback-prize-2021/train_en.csv')

In [18]:
d.reset_index().to_feather('../input/feedback-prize-2021/train_cutlast.fea')

In [19]:
d.describe()

,part,para_id,para_type,start,end,start2,end2,pid,kfold,worker
count,175160.000000,175160.000000,175160.000000,175160.000000,175160.000000,175160.000000,174604.000000,175160.000000,175160.000000,175160.000000
mean,9.023419,5.826250,2.019120,169.021169,206.524583,169.021169,206.352867,87579.500000,2.005446,47.122539
std,5.482194,4.274395,1.736715,162.531740,177.146203,162.531740,177.249046,50564.480913,1.417989,27.455593
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,4.000000,2.000000,1.000000,48.000000,69.000000,48.000000,69.000000,43789.750000,1.000000,23.000000
50%,9.000000,5.000000,2.000000,119.000000,159.000000,119.000000,158.000000,87579.500000,2.000000,47.000000
75%,14.000000,9.000000,3.000000,250.000000,297.000000,250.000000,297.000000,131369.250000,3.000000,71.000000
max,18.000000,31.000000,7.000000,1369.000000,1656.000000,1369.000000,1430.000000,175159.000000,4.000000,94.000000


In [20]:
d.kfold.value_counts()

4    35518
2    35277
0    35253
3    34768
1    34344
Name: kfold, dtype: int64

In [21]:
d.part.value_counts()

15    9810
17    9756
8     9554
1     9495
10    9468
0     9374
11    9374
7     9312
5     9277
16    9271
13    9199
14    9156
2     9151
6     9115
12    8900
3     8827
9     8790
18    8674
4     8657
Name: part, dtype: int64

In [22]:
d.worker.value_counts()

0     2274
75    2143
51    2107
7     2104
6     2102
      ... 
1     1604
2     1593
71    1535
94    1528
19    1521
Name: worker, Length: 95, dtype: int64

In [23]:
len(d.worker == 0)

175160

In [24]:
id = 'C196DA0C7688'
d[d.id==id]

,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,worker
54161,C196DA0C7688,17,0,5,0,24,0,24.0,The author has been studying Venus to find out...,The author has been studying Venus to find out...,54161,0,85
54162,C196DA0C7688,17,1,2,24,185,24,185.0,[BR] [BR] The author says that Venus has many ...,The author has been studying Venus to find out...,54162,0,85
54163,C196DA0C7688,17,2,3,185,222,185,222.0,"humans no matter what. [BR] [BR] Therefore, Ve...",The author has been studying Venus to find out...,54163,0,85


In [25]:
d[d.id==id].text.values[0]

'The author has been studying Venus to find out if humans can live there and there are some reason that humans cant live there. [BR] [BR] The author says that Venus has many feature like earth and that it may be possible for humans to live there. Even though they have seen what could happen like the bad part of living there, they are still trying to find a way to make humans live there. Venus is like Earths "twin" because it is the closest planet to the earth unlike the other planets. Also, Venus atmosphere of carbon dioxide is 97 percent. [BR] [BR] Venus may be worth studying for but despite the danger that it may have to any human could be bad and even though the author says that they are going to have to make humans a part of there studyies so they can find a way to live there. In the text it says that "numerous factors contribute to Venus\'s reputation as a challenging planet for humans to study, despite the proximity to us" saying that they will have to study humans no matter what